# Operator Estimation

Measure observables is a convenient abstraction that allows....

In [38]:
import numpy as np
from pyquil import Program, get_qc
from pyquil.gates import *

## Construct a state with a `Program`


In [64]:
qubits = [0]
prog = Program(H(qubit))
print(prog)

H 1



## Construct a `ObservablesExperiment` and turn it into a list of `Programs`


In [80]:
from pyquil.paulis import *
from forest.benchmarking.operator_estimation import ExperimentSetting, zeros_state,ObservablesExperiment, generate_experiment_programs

# make ExperimentSetting
expt_settings = [ExperimentSetting(zeros_state([0]), pt) for pt in [sI(0), sX(0), sY(0), sZ(0)]]
print('ExperimentSetting:\n==================\n',expt_settings,'\n')

# make an ObservablesExperiment
obs_expt = ObservablesExperiment(expt_settings, program=prog)
print('ObservablesExperiment:\n======================\n',obs_expt,'\n')

# convert it to a list of programs and a list of qubits
expt_progs = generate_experiment_programs(obs_expt, active_reset=False)
print('Programs and Qubits:\n======================\n', expt_progs)

ExperimentSetting:
 [ExperimentSetting[Z0_0→(1+0j)*I], ExperimentSetting[Z0_0→(1+0j)*X0], ExperimentSetting[Z0_0→(1+0j)*Y0], ExperimentSetting[Z0_0→(1+0j)*Z0]] 

ObservablesExperiment:
 H 1
0: Z0_0→(1+0j)*I
1: Z0_0→(1+0j)*X0
2: Z0_0→(1+0j)*Y0
3: Z0_0→(1+0j)*Z0 

Programs and Qubits:
 ([<pyquil.quil.Program object at 0xa167e86a0>, <pyquil.quil.Program object at 0xa162a2390>, <pyquil.quil.Program object at 0xa167e82e8>, <pyquil.quil.Program object at 0xa162a2c18>], [[], [0], [0], [0]])


## PyQuil will run the tomography programs

In [ ]:
from pyquil.operator_estimation import measure_observables

qc = get_qc('2q-qvm')
# Over-write full quilc compilation with a much more simple
# version that *only* substitutes gates to Rigetti-native gates.
# We don't want to accidentally compile away our tomography circuit
# or map to different qubits.
from forest.benchmarking.compilation import basic_compile
qc.compiler.quil_to_native_quil = basic_compile

results = list(measure_observables(qc=qc, tomo_experiment=experiment, n_shots=1_000))
results